### 样本交易特征统计
- 以账期订单为样本

In [3]:
#读取所有文件
import pandas as pd
import os
import numpy as np
import re
from funcs import *
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

In [4]:
#统计每个客户账期订单数据
indexs = ['订单总数','合作时间（月）','最近交易（X月之前）','最近连续交易月数','采购断档总月数','订单时间间隔均值（天）','订单时间间隔方差（天）',
          '交易总量','交易量最大值','交易量最小值','交易量方差','交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值',
          '交易稳定性','供应商数量','采购产品种类数量',
          '账期订单占比','账期订单交易量均值','账期订单交易额均值','账期合作时间（月）','最近一笔账期订单（X月之前）']

In [5]:
#统计所有交易特征
def ac_feature(trade,t_now):
    feature = []
    num = trade.shape[0] #订单数
    coo_month = coo_months(trade[u'创建时间'])  #合作时间（月）
    feature.append(num)
    feature.append(coo_month)
    feature.append(recent_month(trade[u'创建时间'],t_now))
    feature.append(recent_last_month(trade[u'创建时间']))
    
    purchase_month = purchase_months(trade[u'创建时间'])
    gap_months = coo_month-purchase_month  #采购断档总月数
    feature.append(gap_months)
    
    tim_st = time_gap(trade[u'创建时间'])
    feature.append(tim_st[0]) #均值
    feature.append(tim_st[1]) #方差
    
    t_num = trade[u'销售数量（吨）'].agg(['sum','max','min','std','mean'])
    t_money = trade[u'销售金额（元）'].agg(['sum','max','min','std','mean'])
    feature.extend(t_num)
    feature.extend(t_money)
    
    feature.append(trade_stab_all(trade))
    feature.append(trade[u'货物'].apply(lambda x:x.split(' ')[1]).value_counts().shape[0])
    feature.append(trade[u'货物'].apply(lambda x:x.split(' ')[0]).value_counts().shape[0])
        
    zhangqi = trade[trade[u'是否账期']==u'是']
    if zhangqi.shape[0]>0:
        feature.append(zhangqi.shape[0]*1.0/num)
        feature.append(zhangqi[u'销售数量（吨）'].mean())
        feature.append(zhangqi[u'销售金额（元）'].mean())
        feature.append(coo_months(zhangqi[u'创建时间']))
        feature.append(recent_month(zhangqi[u'创建时间'],t_now))
    else:
        feature.append(0)
        feature.append(0)
        feature.append(0)
        feature.append(0)
        feature.append(np.nan)
    return feature

In [6]:
def df_sub(record,trade):
    df = pd.DataFrame(index=indexs)    
    tmps = zip(record[u'应收单号'],record[u'起算日期'])
    for t in tmps:
        trade_sub = trade[trade[u'创建时间']<t[1]]
        if trade_sub.shape[0]>1:
            fs = ac_feature(trade_sub.iloc[1:,:],t[1])
        else:
            fs = [np.nan]*len(indexs)
        df[t[0]]=fs
    return df

In [7]:
#所有账期订单数据统计
zq_files = os.listdir('./data/loan_list/')
#所有订单数据
files = VisitDir('./data/client_data/')
files =[f for f in files if '交易记录' in f]

samples = pd.DataFrame()
for f in zq_files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    record = record.iloc[1:,:]  #账期数据
    for f1 in files:
        if f1.split('/')[-2]==f.split('.')[0]:
            trade = pd.read_excel(f1)   #交易数据
            df = df_sub(record,trade)          
            if samples.shape[0]==0:
                samples=df
            else:
                samples = samples.merge(df,left_index=True,right_index=True,how='left')
            continue

samples =samples.T

In [8]:
samples['采购频率（订单总数/合作月数）'] =samples['订单总数']*1.0/samples['合作时间（月）']
samples['采购聚集程度（订单总数/供应商数）'] =samples['订单总数']*1.0/samples['供应商数量']
samples.to_csv('./data/zq_feature_trade.csv')
samples

,订单总数,合作时间（月）,最近交易（X月之前）,最近连续交易月数,采购断档总月数,订单时间间隔均值（天）,订单时间间隔方差（天）,交易总量,交易量最大值,交易量最小值,...,交易稳定性,供应商数量,采购产品种类数量,账期订单占比,账期订单交易量均值,账期订单交易额均值,账期合作时间（月）,最近一笔账期订单（X月之前）,采购频率（订单总数/合作月数）,采购聚集程度（订单总数/供应商数）
YS20170228152309,18.0,12.0,1.0,2.0,1.0,18.588235,15.544528,556.75,54.00,20.00,...,0.062321,5.0,1.0,0.888889,30.609375,188399.218750,10.0,2.0,1.500000,3.600000
YS20170103150373,16.0,9.0,3.0,9.0,0.0,16.333333,10.668155,502.75,54.00,20.00,...,0.103964,5.0,1.0,0.937500,30.850000,189493.166667,8.0,3.0,1.777778,3.200000
YS20161117147341,15.0,9.0,1.0,9.0,0.0,16.642857,11.000749,448.75,50.00,20.00,...,0.034467,5.0,1.0,0.933333,29.196429,174678.392857,8.0,1.0,1.666667,3.000000
YS20161115147142,15.0,9.0,1.0,9.0,0.0,16.642857,11.000749,448.75,50.00,20.00,...,0.034467,5.0,1.0,0.933333,29.196429,174678.392857,8.0,1.0,1.666667,3.000000
YS20161025145675,14.0,9.0,0.0,9.0,0.0,17.769231,10.576340,422.75,50.00,20.00,...,0.037118,5.0,1.0,0.928571,29.442308,173615.192308,8.0,0.0,1.555556,2.800000
YS20161019110425,14.0,9.0,0.0,9.0,0.0,17.769231,10.576340,422.75,50.00,20.00,...,0.037118,5.0,1.0,0.928571,29.442308,173615.192308,8.0,0.0,1.555556,2.800000
YS20160926108844,12.0,7.0,1.0,7.0,0.0,16.090909,10.064338,346.75,50.00,20.00,...,0.031785,5.0,1.0,0.916667,27.886364,156459.772727,6.0,1.0,1.714286,2.400000
YS20160926108834,12.0,7.0,1.0,7.0,0.0,16.090909,10.064338,346.75,50.00,20.00,...,0.031785,5.0,1.0,0.916667,27.886364,156459.772727,6.0,1.0,1.714286,2.400000
YS20160824106578,11.0,7.0,0.0,7.0,0.0,16.700000,10.392839,315.75,50.00,20.00,...,0.034963,5.0,1.0,0.909091,27.575000,153226.750000,6.0,0.0,1.571429,2.200000
YS20160811171089,10.0,7.0,0.0,7.0,0.0,17.777778,10.413666,284.75,50.00,20.00,...,-0.022263,4.0,1.0,0.900000,27.194444,150101.944444,6.0,0.0,1.428571,2.500000


### 逾期样本订单

In [9]:
#所有逾期记录
files = os.listdir('./data/loan_list/')
loan_all = pd.DataFrame()
for f in files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    loan_all = loan_all.append(record.iloc[1:,:],ignore_index=True)
loan_all['zq_day'] = loan_all[u'账期'].apply(lambda x:re.findall('\d+',x)[0])
loan_all['yq_day'] = loan_all[u'逾期'].apply(lambda x:re.findall('^(-\d+|\d+)',x)[0])
loan_all['zq_day'] = loan_all['zq_day'].astype(int)
loan_all['yq_day'] = loan_all['yq_day'].apply(lambda x:int(x))
loan_all.to_csv('./data/loan_all.csv',index=None)
loan_all=loan_all.drop_duplicates()
loan_all.shape

(1430, 15)

In [10]:
loan_all['4天以内占比']=0
loan_all['5-10天占比']=0
loan_all['11天以上占比']=0
loan_all[u'逾期占比']=0
loan_all[u'提前还款占比']=0
loan_all.shape

(1430, 20)

In [11]:
def yq_sts(list):
    yq_4=0
    yq_10=0
    yq_11=0
    zq_num = len(list)
    for i in list:
        if i>0 and i<=4:
            yq_4+=1
        elif i>4 and i<=10:
            yq_10+=1
        elif i>10:
            yq_11+=1
            
    return yq_4*1.0/zq_num,yq_10*1.0/zq_num,yq_11*1.0/zq_num

In [12]:
#统计
yq_ratio =[]
tq_ratio =[]
yq_day_ratio =[]
yq_avg =[]
loan_all[u'起算日期'] =pd.to_datetime(loan_all[u'起算日期'])
for i in xrange(loan_all.shape[0]):
    name = loan_all.iat[i,1]
    time = loan_all.iat[i,10]
    loan_sub = loan_all[(loan_all[u'客户'] == name) & (loan_all[u'起算日期']<time)]
    if loan_sub.shape[0]>0:
        tmps = yq_sts(loan_sub['yq_day'])
        yq_day_ratio.append(tmps)
        yq_ratio.append(loan_sub[loan_sub['yq_day']>0].shape[0]*1.0 / loan_sub.shape[0])
        tq_ratio.append( loan_sub[loan_sub['yq_day']<0].shape[0]*1.0 / loan_sub.shape[0])
        if loan_sub[loan_sub['yq_day']>0].shape[0]>0:
            yq_avg.append( loan_sub[loan_sub['yq_day']>0]['yq_day'].mean())
        else:
            yq_avg.append(0)
    else:
        yq_day_ratio.append((np.nan,np.nan,np.nan))
        yq_ratio.append(np.nan)
        tq_ratio.append(np.nan)
        yq_avg.append(np.nan)

loan_all['4天以内占比']=[x[0] for x in yq_day_ratio]
loan_all['5-10天占比']=[x[1] for x in yq_day_ratio]
loan_all['11天以上占比']=[x[2] for x in yq_day_ratio]
loan_all[u'逾期占比']=yq_ratio
loan_all[u'提前还款占比']=tq_ratio
loan_all[u'平均逾期天数']=yq_avg
loan_all.to_csv('./data/loan_all.csv',index=None)
loan_all.head()

,业务类型,客户,已收金额,应收,应收余额,应收单号,应收总额,应收日期,收款状态,账期,...,逾期,销售合同号,zq_day,yq_day,4天以内占比,5-10天占比,11天以上占比,逾期占比,提前还款占比,平均逾期天数
0,账期业务,佛山市南海区松岗明珠塑料有限公司,184520.0,NaN,0.0,YS20170228152309,184520.0,2017-03-30,已收款,30天,...,11天,ZSL20170227LL-C003WJ,30,11,0.157895,0.0,0.0,0.157895,0.263158,1.333333
1,账期业务,佛山市南海区松岗明珠塑料有限公司,171990.0,NaN,0.0,YS20170103150373,171990.0,2017-01-13,已收款,10天,...,0天,ZSL20161228LL-C041WJ,10,0,0.166667,0.0,0.0,0.166667,0.277778,1.333333
2,账期业务,佛山市南海区松岗明珠塑料有限公司,198450.0,NaN,0.0,YS20161117147341,198450.0,2016-12-02,已收款,15天,...,-2天,ZSL20161025LL-C009WJ,15,-2,0.176471,0.0,0.0,0.176471,0.235294,1.333333
3,账期业务,佛山市南海区松岗明珠塑料有限公司,198450.0,NaN,0.0,YS20161115147142,198450.0,2016-11-29,已收款,15天,...,0天,ZSL20161025LL-C009WJ,15,0,0.187500,0.0,0.0,0.187500,0.250000,1.333333
4,账期业务,佛山市南海区松岗明珠塑料有限公司,186940.0,NaN,0.0,YS20161025145675,186940.0,2016-11-09,已收款,15天,...,0天,ZSL20161011LL-C038WJ,15,0,0.200000,0.0,0.0,0.200000,0.266667,1.333333
